# 딥러닝 모델 서빙(Gunicorn Flask RabbitMQ Celery tensorflowserving)

### gunicorn - Flask

#### Flask 서버

In [ ]:
# flask_server.py
from flask import Flask, request, render_template
from PIL import Image
import numpy as np
from serving_worker.serving import serving_task
from base64 import b64encode
import json

from serving_worker.tasks import celery_worker
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/serving',methods=['GET','POST'])
def serving():
    print(request.method)
    if request.method == 'POST':
        data = request.files['chooseFile']
        data_byte = data.read()
        data_base64 = b64encode(data_byte).decode()
        
        result = serving_task.delay(data_base64)
        TASK_ID = result.id
        return render_template('progress.html', TASK_ID = TASK_ID)

    return render_template('serving.html')

@app.route('/progress',methods=['GET','POST'])
def progress():
    print(request.method)
    if request.method == 'POST':
        TASK_ID = request.form['TASK_ID']

        task = celery_worker.AsyncResult(TASK_ID)
        print("시작")
        print(task)
        print(task.ready())
        print(task.state)
        state = task.state
        if state == 'SUCCESS':
            print(task.get())
            predict = task.get()
            task.forget()
        
            return json.dumps({'state':state,'predict':predict})
        return json.dumps({'state':state})

    return render_template('progress.html',RESULT_ID=5)

if __name__ == '__main__':
    app.run(debug=True,host='0.0.0.0',port='8000')

In [ ]:
gunicorn flask_server:app --bind=0.0.0.0:8000 -w 4

### RabbitMQ - Celery - tensorflow/serving

#### RabbitMQ 서버

In [ ]:
# Mac 기준
# /usr/local/sbin
./rabbitmq-server

#### Celery 서버
- broker : rabbitmq
- backend: sqlite

In [ ]:
# serving_worker/tasks.py
from celery import Celery 

celery_worker = Celery(
    'serving_worker.tasks',
    backend='db+sqlite:///results.sqlite',
    broker='amqp://guest@localhost//',
    include=['serving_worker.serving'])

#### Celery - Tensorflow/serving
- tensorflow 서버로 이미지 보내는 코드

In [ ]:
# serving_worker/serving.py
from serving_worker.tasks import celery_worker
from PIL import Image 
import numpy as np
from base64 import b64decode
from io import BytesIO
import requests
import json 

address = 'http://changminkimserver29672.iptime.org:8501/v1/models/mnist:predict'

@celery_worker.task
def serving_task(data_base64):
    data_byte = b64decode(data_base64)
    data_buffer = BytesIO(data_byte)
    image = Image.open(data_buffer)
    pre_image = (np.array(image).reshape((1,28,28,1)) - 127.5)/127.5
    pre_image = pre_image.tolist()
    data = {'instances':pre_image}
    data = json.dumps(data)

    result = requests.post(address,data)
    data = result.content
    data = json.loads(data)
    predict = np.argmax(data['predictions'])
    print(predict)
    return int(predict)

In [ ]:
celery -A serving_worker.tasks worker --loglevel=info